### ДЗ - определение ССЗ

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [1]:
import seaborn as sns
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,  mean_squared_error, log_loss
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
#from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin


In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделим наши данные на тренировочную и тестовую выборки

In [3]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [6]:
def fit_predict(clfr = None):
    if clfr == None:
        clfr = LogisticRegression(random_state = 42) 

    classifier = Pipeline([
        ('features',feats),
        ('classifier', clfr),
    ])

    #запустим кросс-валидацию
    print("Кросс-валидация ...")
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=3, scoring='roc_auc')
    cv_score_mean = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    print("Обучение на всем тренировочном наборе ...")
    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    print("Предсказание ...")
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    r={
        'cv_scores': cv_scores,
        'cv_score_mean': cv_score_mean,
        'cv_score_std': cv_score_std,
        'y_score': y_score
    }

    return r



In [7]:
def get_metrics(r, metric_name, threshold):
    
    if 'metrics' not in r:
        r['metrics'] = {}
    r[metric_name] = {}
    d = r['metrics'][metric_name] = {}
    d['best_threshold'] = threshold
    y_score = r['y_score']
    d['roc_auc_score'] = roc_auc_score(y_true=y_test, y_score=y_score)
    d['log_loss_score'] = log_loss(y_true=y_test, y_pred=y_score)
    
    d['confusion_matrix'] = cnf_matrix = confusion_matrix(y_test, y_score > threshold)

    TN = cnf_matrix[0][0]
    FN = cnf_matrix[1][0]
    TP = cnf_matrix[1][1]
    FP = cnf_matrix[0][1]

    d['TPR']= TP/(TP+FN)
    d['FPR'] = FP/(FP+TN)
    d['TNR'] = TN/(FP+TN)

    d['precision'] = precision = TP / (TP + FP)
    d['recall'] = recall = TP / (TP + FN)

    b=1
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    d['fscore'] = fscore
    
    return r

def get_metrics_fscore(r):
    y_score = r['y_score']
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    b=1
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    return get_metrics(r, 'fscore', thresholds[np.argmax(fscore)])

def get_metrics_business(r, rubl_to_test = 3000, rubl_to_early_treatment = 20000, rubl_to_late_treatment = 30000):
    costs = []
    metric = []
    thresholds = np.linspace(0, 1, 20).tolist()
    y_score = r['y_score']

    for t in thresholds:
        cnf_matrix = confusion_matrix(y_test, y_score > t)
        rubl_no_tests = (cnf_matrix[1][1]+cnf_matrix[1][0]) * rubl_to_late_treatment
        rubl_all_test = np.sum(cnf_matrix) * rubl_to_test + (cnf_matrix[1][0]+cnf_matrix[1][1]) * rubl_to_early_treatment
        rubl_ML = (cnf_matrix[1][1]+cnf_matrix[0][1]) * rubl_to_test + cnf_matrix[1][0] * rubl_to_late_treatment + cnf_matrix[1][1] * rubl_to_early_treatment
        metric.append(rubl_all_test - rubl_ML)
        costs.append((rubl_no_tests, rubl_all_test, rubl_ML))

    print(metric)
        
    ix = np.argmax(metric)
    r = get_metrics(r, 'business', thresholds[ix])
    r['metrics']['business']['notes']= [f"Выгода: {metric[ix]}; Расходы: без тестирования: {costs[ix][0]}, с полным тестированием: {costs[ix][1]}, с ML: {costs[ix][2]}"]
    return r

def print_metrics(r):
    print(f"Cross-validation score: {r['cv_score_mean']:.3f}+-{r['cv_score_std']:.3f}")
    for mn in r['metrics']:
        d = r['metrics'][mn]
        print(f"Metrics (optimal threshold for {mn} = {d['best_threshold']:.3f}): ")
        print(f"    fscore: {d['fscore']:.3f}, precision: {d['precision']:.3f}, recall: {d['recall']:.3f}, ROC AUC: {d['roc_auc_score']:.3f}, Log loss: {d['log_loss_score']:.3f}")
        print(f"    TPR: {d['TPR']:.3f}, FPR: {d['FPR']:.3f}, TNR: {d['TNR']:.3f}")
        if 'notes' in d:
            for n in d['notes']:
                print(f"    {n}")
        print(f"    Confusion matrix: ")
        print(f"        ", d['confusion_matrix'][0])
        print(f"        ", d['confusion_matrix'][1])
    return r
        


In [8]:
def draw_roc_auc(r):
    sns.set(font_scale=1.5)
    sns.set_color_codes("muted")

    plt.figure(figsize=(10, 8))
    fpr, tpr, thresholds_ = roc_curve(y_test, r['y_score'], pos_label=1)
    lw = 2
    plt.plot(fpr, tpr, lw=lw, label='ROC curve ')
    plt.plot([0, 1], [0, 1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.savefig("ROC.png")
    plt.show()
    return r
    


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import catboost as catb

results = {}
results['LogisticRegression'] = fit_predict(LogisticRegression(random_state = 42))
results['RandomForest'] = fit_predict(RandomForestClassifier(random_state = 42))
results['CatBoost'] = fit_predict(catb.CatBoostClassifier(silent=True, random_state = 42))



Кросс-валидация ...
Обучение на всем тренировочном наборе ...
Предсказание ...
Кросс-валидация ...
Обучение на всем тренировочном наборе ...
Предсказание ...
Кросс-валидация ...
Обучение на всем тренировочном наборе ...
Предсказание ...


In [10]:
for classifier_name in results:
    print('-------------------------' + classifier_name + '-------------------------')
    print_metrics(get_metrics_business(get_metrics_fscore(results[classifier_name])))


-------------------------LogisticRegression-------------------------
[0, -125000, 251000, 1282000, 2665000, 3991000, 4812000, 4810000, 3638000, 1848000, -1413000, -4830000, -8737000, -13212000, -17790000, -22453000, -26845000, -30612000, -33325000, -34300000]
Cross-validation score: 0.786+-0.004
Metrics (optimal threshold for fscore = 0.387): 
    fscore: 0.730, precision: 0.647, recall: 0.837, ROC AUC: 0.784, Log loss: 0.578
    TPR: 0.837, FPR: 0.449, TNR: 0.551
    Confusion matrix: 
         [4861 3959]
         [1411 7269]
Metrics (optimal threshold for business = 0.316): 
    fscore: 0.719, precision: 0.594, recall: 0.911, ROC AUC: 0.784, Log loss: 0.578
    TPR: 0.911, FPR: 0.613, TNR: 0.387
    Выгода: 4812000; Расходы: без тестирования: 260400000, с полным тестированием: 226100000, с ML: 221288000
    Confusion matrix: 
         [3410 5410]
         [ 774 7906]
-------------------------RandomForest-------------------------
[119000, 1200000, 2445000, 3344000, 3924000, 3943000, 

### Отчет и выводы

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

Использовались LogisticRegression, RandomForestClassifier, CatBoostClassifier с фиксированным random_state и параметрами по умолчанию.

2. при обучении моделей обязательно использовать кроссвалидацию

Использовалась 

3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

Выведены все метрики, в том числе бизнес-метрика "выгода от предсказания ССЗ с помощью МО по сравнению с полным тестированием". Как ожидалась, оптимизация под бизнес-метрику давала разумный результат и выгоду при сравнительно высокой цене тестов. При низкой цене тестов выгоднее было протестировать всех.

4. сделать выводы о том, какая модель справилась с задачей лучше других

Лучше всего справилась модель CatBoostClassifier, по fscore, ROC-AUC и бизнес-метрике. Интересно, что простая модель LogicsticRegression оказалась на втором месте. Но сравнение не очень объективное, так как настройка параметров не производилась, использовались параметры по умолчанию. 

5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?